In [18]:
!apt-get install -y libzbar0
!pip install --quiet opencv-python python-barcode pyzbar ipywidgets

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libzbar0 is already the newest version (0.23.92-4build2).
0 upgraded, 0 newly installed, 0 to remove and 122 not upgraded.


In [30]:
import pandas as pd
import numpy as np
import cv2
from pyzbar.pyzbar import decode
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from ipywidgets import FileUpload, Button, VBox, Output
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)


# Loading dataset
df = pd.read_csv("/kaggle/input/world-food-facts/en.openfoodfacts.org.products.tsv", 
                 sep='\t', low_memory=False)

# Keeping essential columns and shortening dataset
df = df[['code', 'product_name', 'brands', 'ingredients_text', 
         'carbohydrates_100g', 'fat_100g', 'fiber_100g', 
         'proteins_100g', 'salt_100g', 'sugars_100g']].dropna(subset=['code', 'product_name'])
df = df[df['code'].apply(lambda x: str(x).isdigit())].head(10_000)


In [80]:
def lookup_barcode(barcode):
    product = df[df['code'] == str(barcode)]
    if not product.empty:
        data = product.iloc[0]
        print(f"📦 Product       : {data['product_name']}")
        print(f"🏢 Brand         : {data.get('brands', 'Not listed')}")
        print(f"🧾 Ingredients   : {data.get('ingredients_text', 'Not listed')}")
        print(f"🍞 Carbs (100g)  : {data.get('carbohydrates_100g', 'Not listed')}")
        print(f"💪 Fat (100g)    : {data.get('fat_100g', 'Not listed')}")
        print(f"🌾 Fiber (100g)  : {data.get('fiber_100g', 'Not listed')}")
        print(f"🥩 Protein (100g): {data.get('proteins_100g', 'Not listed')}")
        print(f"🧂 Salt (100g)   : {data.get('salt_100g', 'Not listed')}")
        print(f"🍬 Sugar (100g)  : {data.get('sugars_100g', 'Not listed')}")
    else:
        print("❌ Barcode not found.")


In [81]:
def detect_and_lookup_barcode(image_data):
    image_array = np.frombuffer(image_data, np.uint8)
    image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

    if image is None:
        print("❌ Failed to decode image.")
        return

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    barcodes = decode(gray)

    if not barcodes:
        print("🚫 No barcode detected.")
        return

    barcode_data = barcodes[0].data.decode("utf-8")
    print(f"📷 Detected Barcode: {barcode_data}")
    
    (x, y, w, h) = barcodes[0].rect
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.putText(image, f"{barcode_data}", (x, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(image_rgb)
    plt.axis('off')
    plt.show()

    lookup_barcode(barcode_data)


In [82]:
# Widgets
upload_widget = FileUpload(accept='.png', multiple=False)
process_button = Button(description="Decode & Lookup")
output = Output()

def on_button_click(b):
    with output:
        output.clear_output()
        if not upload_widget.value:
            print("❌ Please upload a PNG image.")
            return
            
        uploaded_file = upload_widget.value[0] 
        image_data = uploaded_file['content']  
        
        print(f"📤 Processing: {uploaded_file['name']}")
        detect_and_lookup_barcode(image_data)

# Attaching event
process_button.on_click(on_button_click)

# Displaying the UI
display(VBox([upload_widget, process_button, output]))